<a href="https://colab.research.google.com/github/luciabarg/mentoria_img/blob/master/4-Aprendizaje%20Supervisado/6_Aprendizaje_Supervisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación </h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
<h2>Mentoría: Detección de objetos en imágenes</h2>
</center>


En la siguiente notebook, incorporaremos parte de lo realizado en las entregas 1 de los grupos de estudiantes: 

* [Grupo 1](https://github.com/Ana1890/Deteccion-de-objetos-en-imagenes) : Ana Rusconi, Ignacio Grosso y  Mariano Lucero

* [Grupo 2](https://github.com/leorodriguez/mentoria-diplodatos-grupo2):  Diego Gonzalez, Florencia Caballero, Joel Vivas y Leonardo Rodriguez


#### <a name="generacion_tabla"></a>
#### **Análisis general del dataset**

En las entregas anteriores tuvieron la oportunidad de explorar una pequeña muestra de las imágenes para que el análisis sea mas representativo del conjunto de datos total, en esta entrega utilizaremos el dataset completo.

En esta notebook utilizaremos la tabla generada en'[5-Generación_de_fotos_recortadas_y_tabla-Aprendizaje_Supervisado](https://colab.research.google.com/drive/1fgB4F4DlBqkCvOatqo3whm4ZTPsiP0L7?usp=sharing)'

La carpeta con los datos, es compartida en el [Drive]( ) y en el desarrollo de esta entrega con las consignas, se utilizará la misma en el entorno de Colab, pero también pueden descargarla desde el repositorio de Github de la mentoría en la carpeta ['dataset_crop'](https://github.com/luciabarg/mentoria_img).

Una pequeña cantidad de fotos las guardé como test. Que se las daré al final. 

La siguiente notebook usa como entorno Colab:

In [8]:
#vinculamos el Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# librerías necesarias 
import os
from IPython.display import Image as Imge
import pandas as pd
import numpy as np
import glob

#seguimos agregando librerías necesarias:
#las que no tengan, vayan instalándolas
import cv2
import numpy as np

import matplotlib.pyplot as plt
import PIL
from PIL import Image
from os import path

from google.colab.patches import cv2_imshow #en Colab!

import seaborn as sns

import re

import json


from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


Para que se hagan una idea de la cantidad de datos que deberíamos analizar, los exploramos y visualizamos con una tabla:

In [13]:
#volvemos a la carpeta classes, vemos de copiar todos los csv correspondientes a esta carpeta
os.chdir('/content/drive/My Drive/Mentoria/crop')
#listamos solo las carpetas
! ls -d */
! ls -d */*/

Amarillos/  Mano-Amarillo/  Manos/  Mano-Verde/  Verdes/
Mano-Amarillo/Amarillo/  Mano-Verde/Mano/
Mano-Amarillo/Mano/	 Mano-Verde/Verde/


#### <a name="generacion_tabla"></a>
#### **Análisis general de la tabla**

1.   Cuántas fotos originales hay? (no filas! fotos :) )
2.   Separar los tamaños 'P' y 'G'. Tratarlos como conjuntos de datos por separado. 
      


In [15]:
#leemos el archivo de la tabla:
csv_wlabels_df = pd.read_csv('csv_wlabels_all_df.csv', index_col=0)
csv_wlabels_df = csv_wlabels_df.drop(['Type'], axis=1)  #la eliminamos ya que es una división previa de los datos, que no utilizaremos en este caso.
print(csv_wlabels_df.shape)
csv_wlabels_df.head()

(15197, 2060)


,filename,xmin,xmax,ymin,ymax,Time,Size,Hand_Pixel_Width,Hand_Pixel_Height,Area,guantes,color,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,M_G_Mostrando_Amarillos_0_double_box1.jpg,218,461,464,715,M,G,243,251,60993,si,Amarillo,0.740974,1.066019,0.132695,0.224892,0.140456,0.129134,0.201447,1.076733,0.461108,0.299800,0.457940,0.201904,0.569677,0.737381,0.480929,0.200076,0.018410,0.674050,0.028826,0.005310,0.128693,0.592353,0.138541,0.141684,0.363294,0.264674,0.519292,0.292655,...,0.463567,0.091676,0.300955,0.696112,2.409413,0.536980,0.102273,0.157441,0.165093,1.067619,0.751934,0.231443,0.487437,0.446320,0.822407,0.527523,0.217110,0.489194,1.535306,0.521390,0.599480,0.201182,0.391337,0.040206,1.347440,0.448389,0.462778,0.590751,0.877142,0.009986,0.939221,0.859877,0.981910,0.356366,1.748501,0.288037,0.079172,0.266234,0.078312,0.421612
1,M_G_Mostrando_Amarillos_0_double_box2.jpg,617,865,432,598,M,G,248,166,41168,si,Amarillo,1.621832,0.208859,0.627041,0.061717,0.052486,0.064176,0.479256,1.068516,0.509059,0.207385,0.218195,0.169190,0.695486,0.237641,0.146711,0.126334,0.029900,0.148363,0.126833,0.181351,0.044585,0.447270,0.134360,0.113650,1.153900,0.653832,0.041024,0.543086,...,0.816892,0.027489,0.183116,0.626535,1.734605,0.672962,0.030057,0.101685,0.295544,0.392251,0.874054,0.105285,0.110713,0.421417,1.335695,0.500971,1.139208,0.110803,0.850265,0.618181,0.077352,0.376629,0.319234,0.783792,0.427963,0.133063,0.064981,0.017857,0.320706,0.219192,1.199712,0.808466,0.491182,0.356616,0.812572,0.481357,0.071471,0.572322,0.052072,0.213453
2,M_G_Mostrando_Amarillos_100_double_box1.jpg,831,1138,256,408,M,G,307,152,46664,si,Amarillo,1.029100,0.246964,0.825601,0.460009,0.467037,0.339540,0.653041,1.324649,0.890402,0.488183,0.639772,0.085819,0.928661,0.148958,0.066155,0.125391,0.191808,0.506981,0.257565,0.073959,0.115524,0.552251,0.764911,0.047104,0.207601,0.126297,0.286010,0.733891,...,0.367434,0.436251,0.000000,0.863331,0.112969,0.902164,0.040710,0.467132,0.242757,0.853286,0.677067,0.300664,0.587456,0.112658,1.077973,0.144736,0.294853,0.065939,0.708062,1.192815,0.317312,0.421216,0.425882,1.091190,0.595093,0.138662,0.045023,0.543350,0.155850,0.305535,0.463963,0.355296,0.360880,0.322859,0.917611,0.408334,0.733588,0.416891,0.151471,0.045931
3,M_G_Mostrando_Amarillos_100_double_box2.jpg,511,812,8,228,M,G,301,220,66220,si,Amarillo,0.086092,0.455828,0.418580,0.131673,0.437982,0.424093,0.350489,1.974670,0.183887,0.241193,0.046988,0.312450,0.497026,0.135973,0.195146,0.010869,0.109539,0.234128,0.235778,0.108441,0.054660,0.327069,0.439913,0.050670,0.159292,0.159228,0.311495,0.999218,...,0.625021,0.219692,0.029219,0.496582,1.540741,2.338270,0.039844,0.174714,0.383931,0.449525,0.104648,0.205243,0.104140,0.109657,1.092486,0.087052,0.451129,0.034648,0.272219,0.209507,0.138235,0.193764,0.081227,1.006930,1.506502,0.044992,1.047619,0.605296,0.391379,0.164156,0.325708,0.028549,0.225465,0.325055,0.460382,0.127341,0.140412,0.157423,0.296102,0.023314
4,M_G_Mostrando_Amarillos_101_double_box1.jpg,668,988,0,222,M,G,320,222,71040,si,Amarillo,0.068881,0.185060,0.427007,0.637546,0.357156,0.125816,0.450526,1.246901,0.236921,0.393104,0.632759,0.413947,0.119917,0.192305,0.372677,0.075668,0.103642,0.165934,0.239047,0.054987,0.129182,0.178563,0.217686,0.130172,0.080214,0.346873,0.392784,1.042759,...,0.205059,0.225329,0.062677,0.558553,1.780167,1.919532,0.165511,0.649451,0.168220,0.127490,0.310012,0.071468,0.196005,0.042177,1.261848,0.216220,0.501204,0.194764,0.183244,0.426975,0.477355,0.349516,0.131819,0.489718,0.981365,0.049099,0.471963,0.190929,0.413278,0.175050,0.523058,0.028629,0.076427,0.462088,0.694069,0.060703,0.065414,0.296513,0.368063,0.039469


Preguntas antes de entrenar los algoritmos:

*   Identifique las clases y cuántas hay de cada una (tanto binaria como multiclase). Hay desbalanceo? Si lo hubiera, cómo lo solucionarían? 

*   Un par de preguntas con respecto la separación de los datos:
     
     * Qué se hace con las columnas *'target'*? ('guantes' y 'color'). Hay columnas que pueden ser descartadas? Si quisiéramos aprovechar la información de las columnas descartadas, como podríamos hacerlo? 
    
* Para la variable target elegida, conviene dejarla así o hay que hacer alguna transformación?
* Conviene normalizar antes de pasar los datos al algoritmo?

### Entrenamiento de algoritmos para generar un modelo:

* Separar los datos 'P' y 'G' en conjuntos de test y entrenamiento en un 70-30. Recordar que tenemos para hacer tanto clasificación binaria como multiclase.

* Entrenar los algoritmos de clasificación KNN, SMV y Random Forest:
  * Entrenar el modelo sin parámetros y tomarlo como baseline. Revisar documentación para identificar los parámetros correspondientes.
  * Utilizar CV para encontrar los parámetros para encontrar el mejor modelo de cada uno de ellos. (Cuál métrica van a utilizar? Porqué eligen esa en particular? Se puede utilizar la misma métrica para toda clase de datos? )
  * Hacer una tabla para comparar los mejores modelos de cada algoritmo. 
  * Visualizar los resultados por medio de:
    *  Matriz de Confusión
    *  Reporte de Clasificación
  * Recordar de documentar los pasos que están llevando a cabo, comentar el código, plantear inconvenientes y contar lo que encontraron, poner conclusiones :).
  
  Opcional: 
  * Graficar la métrica elegida con respecto al cambio de los parámetros de los conjuntos 'test' y 'train'.
  * Generar las Curvas de ROC correspondientes para los mejores modelos de clasificación binaria.
  * Podemos utilizar los mejores modelos para generar uno mas mejor? :P
  (chusmear: ['Ensembles'](https://www.iartificial.net/ensembles-voting-bagging-boosting-stacking/) )